write a code that read data from ticker.txt file in which the stock ticker symbol is situated , the this code goes to data folder inside which there are two folder first is stock_news_data and other one is stock_price_data . inside the stock_news data there is are .csv file name like f"{stock_symbol}_sentiments.csv" where the data is like date,mean_sentiment
2024-01-01,0.42857142857142855
2024-01-02,0.32634920634920633
2024-01-03,-0.10386363636363635
2024-01-04,0.15928571428571428
2024-01-05,0.16
2024-01-06,0.15000000000000002 . so now read this data as a dataframe read all the files and then there is other folder named stock_price_data go inside it in there there will be bunch of .json data file . with names f"{stock_symbol}_data.json" the data inside it look like this {
    "ticker": "AAPL",
    "queryCount": 791,
    "resultsCount": 791,
    "adjusted": true,
    "results": [
        {
            "v": 143285672.0,
            "vw": 129.7326,
            "o": 133.52,
            "c": 129.41,
            "h": 133.6116,
            "l": 126.76,
            "t": 1609736400000,
            "n": 1310217
        },
        {
            "v": 97664898.0,
            "vw": 130.7179,
            "o": 128.89,
            "c": 131.01,
            "h": 131.74,
            "l": 128.43,
            "t": 1609822800000,
            "n": 707577
        },..
where symbols stand for c*number
The close price for the symbol in the given time period.

h*number
The highest price for the symbol in the given time period.

l*number
The lowest price for the symbol in the given time period.

ninteger
The number of transactions in the aggregate window.

o*number
The open price for the symbol in the given time period.

otcboolean
Whether or not this aggregate is for an OTC ticker. This field will be left off if false.

t*integer
The Unix Msec timestamp for the start of the aggregate window.

v*number
The trading volume of the symbol in the given time period.

vwnumber
The volume weighted average price.

So loop over this data  and make a pandas df out of it and merge with the df we got from the for the setiment file after converting both of their index to datetime format . The final output should look like this . date , Volume,Volume Weighted,Open,Close,High,Low,Count,mean_sentiment
2024-01-01,87493432.0,43.7151,43.54,43.5575,43.9675,43.4875,158854,0.20653064574314578

In [92]:
import os
import pandas as pd
import json
from datetime import datetime


def read_sentiment_data(stock_symbol):
    file_path = f"data/stock_news_data/{stock_symbol}_sentiments.csv"
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        df["date"] = pd.to_datetime(df["date"])
        df.set_index("date", inplace=True)
        return df
    else:
        print(f"No sentiment data found for {stock_symbol}.")
        return pd.DataFrame()


def read_price_data(stock_symbol):
    file_path = f"data/stock_price_data/{stock_symbol}_data.json"
    if os.path.exists(file_path):
        with open(file_path, "r") as file:
            data = json.load(file)
            results = data.get("results", [])
            df = pd.DataFrame(results)
            df["date"] = pd.to_datetime(df["t"], unit="ms")
            df.set_index("date", inplace=True)
            df.drop(columns=["t"], inplace=True)
            # Rename columns to full form
            df.rename(
                columns={
                    "v": "Volume",
                    "vw": "Volume Weighted",
                    "o": "Open",
                    "c": "Close",
                    "h": "High",
                    "l": "Low",
                    "n": "Count",
                },
                inplace=True,
            )
            return df
    else:
        print(f"No price data found for {stock_symbol}.")
        return pd.DataFrame()


def merge_data(stock_symbols):
    for stock_symbol in stock_symbols:
        final_df = pd.DataFrame()
        sentiment_df = read_sentiment_data(stock_symbol)
        price_df = read_price_data(stock_symbol)
        if not sentiment_df.empty and not price_df.empty:
            # Resample price data to daily frequency to align with sentiment data
            price_df = price_df.resample("D").mean()
            # Merge sentiment and price data
            merged_df = pd.merge(
                sentiment_df, price_df, how="inner", left_index=True, right_index=True
            )
            # Remove rows with data of the current day
            today_date = datetime.today().strftime("%Y-%m-%d")
            merged_df = merged_df[merged_df.index != today_date]
            # Fill short gaps with forward fill
            merged_df = merged_df.ffill(
                limit=3
            )  # Limit defines the maximum number of consecutive NaN values to fill
            # Interpolate longer gaps
            merged_df.interpolate(
                method="linear", inplace=True, limit_area="inside"
            )  # Using linear interpolation
            # Save merged data to a separate file for each ticker
            save_data(merged_df, "data", stock_symbol)


def save_data(data, folder_path, stock_symbol):
    folder_path = os.path.join(folder_path, "production")
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    file_path = os.path.join(folder_path, f"{stock_symbol}_merged_data.csv")
    data.to_csv(file_path)


def main_production():
    try:
        with open("ticker.txt", "r") as ticker_file:
            stock_symbols = [ticker.strip() for ticker in ticker_file.readlines()]
            merge_data(stock_symbols)
            print("Data saved successfully.")
    except FileNotFoundError:
        print("Ticker file not found.")
    except Exception as e:
        print(f"An error occurred: {e}")


if __name__ == "__main__":
    main_production()

Data saved successfully.


This handle misssing values . 

In [90]:
import pandas as pd

# Read the CSV file
file_path = r"D:\Codebase2.0\freelancing\Production_stock_prediction\data\production\AAPL_merged_data.csv"
data = pd.read_csv(file_path)

# Convert 'date' column to datetime format
data["date"] = pd.to_datetime(data["date"])

# Define the range of dates for your dataset
start_date = pd.to_datetime("2021-04-22")
end_date = pd.to_datetime("today")

# Generate a sequence of dates covering the range of your dataset
all_dates = pd.date_range(start=start_date, end=end_date)

# Find missing dates
missing_dates = all_dates[~all_dates.isin(data["date"])]

# Print the missing dates
print("Missing dates:")
print(missing_dates)

Missing dates:
DatetimeIndex(['2024-02-27'], dtype='datetime64[ns]', freq='D')
